In [31]:
# Importar librerías necesarias
import ee
import geemap
import folium
import ipywidgets as widgets


# Inicializar Earth Engine
ee.Initialize(project='buoyant-site-454305-j8')

# Cargar la versión actualizada de MODIS Land Cover (061)
modis = ee.ImageCollection("MODIS/061/MCD12Q1").select('LC_Type1')

In [32]:
# Definir la región de África
africa_roi = ee.Geometry.Polygon([
    [-20, 40], [55, 40], [55, -40], [-20, -40], [-20, 40]  # Coordenadas de África
])

# Definir colores y etiquetas según MODIS Land Cover Classification
legend_colors = [
    "#05450a", "#086a10", "#54a708", "#78d203", "#009900",
    "#c6b044", "#dcd159", "#dade48", "#fbff13", "#b6ff05",
    "#27ff87", "#c24f44", "#a5a5a5", "#ff6d4c", "#69fff8",
    "#f9ffa4", "#1c0dff"
]

legend_labels = [
    "Bosque de hoja perenne",
    "Bosque caducifolio",
    "Bosque mixto",
    "Matorral",
    "Vegetación herbácea",
    "Tierras de cultivo",
    "Áreas urbanas",
    "Pastizales",
    "Zonas húmedas",
    "Cuerpos de agua",
    "Nieve/Hielo",
    "Desierto",
    "Terreno desnudo",
    "Sabanas",
    "Manglares",
    "Arrozales",
    "Otras coberturas"
]

# Crear el mapa interactivo centrado en África
Map = geemap.Map(center=[10, 20], zoom=3)  # Centrado en África

# Agregar la leyenda UNA SOLA VEZ al inicio
Map.add_legend(title="MODIS Land Cover", labels=legend_labels, colors=legend_colors)

# Crear un diccionario de imágenes por año
years = list(range(2001, 2024))  # De 2001 a 2023
modis_dict = {}

for year in years:
    img = modis.filter(ee.Filter.calendarRange(year, year, 'year')).first()
    modis_dict[year] = img.clip(africa_roi)

# Crear un control deslizante (slider) para seleccionar el año
slider = widgets.IntSlider(min=2001, max=2023, step=1, value=2023, description="Año:")

# Función para actualizar el mapa cuando cambia el slider
def update_map(year):
    Map.layers = []  # Limpiar capas anteriores (NO la leyenda)

    # Obtener la imagen del año seleccionado
    img = modis_dict[year]

    # Aplicar máscara para ocultar el resto del mundo
    mask = img.neq(0)
    img_masked = img.updateMask(mask)

    # Parámetros de visualización
    vis_params = {'min': 1, 'max': 17, 'palette': legend_colors}

    # Agregar capa al mapa
    Map.addLayer(img_masked, vis_params, f"MODIS Land Cover {year}")

# Conectar el slider con la función
widgets.interactive(update_map, year=slider)

# Mostrar mapa y slider
display(slider)
Map

IntSlider(value=2023, description='Año:', max=2023, min=2001)

Map(center=[10, 20], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [33]:
# Cargar el dataset CHIRPS (Precipitación mensual desde 1981)
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD").select("precipitation")

# Definir la región de África
africa_roi = ee.Geometry.Polygon([
    [-20, 40], [55, 40], [55, -40], [-20, -40], [-20, 40]  # Coordenadas de África
])

# Crear un diccionario para almacenar imágenes de CHIRPS por año
years = list(range(2003, 2024))  # 2003 a 2023
chirps_dict = {}

for year in years:
    # Filtrar los datos de CHIRPS para cada año y calcular la precipitación anual
    annual_precip = chirps.filter(ee.Filter.calendarRange(year, year, 'year')).sum()
    chirps_dict[year] = annual_precip.clip(africa_roi)

# Crear el mapa interactivo centrado en África
Map = geemap.Map(center=[10, 20], zoom=3)

# Definir parámetros de visualización (mm de precipitación)
vis_params = {
    "min": 0, "max": 2000,  # Valores típicos de precipitación anual en mm
    "palette": ["#ffffff", "#b3ccff", "#6699ff", "#3366ff", "#0033ff",
                "#009933", "#33cc33", "#ffff00", "#ff9900", "#ff3300", "#990000"]
}

# Agregar la leyenda para precipitación CHIRPS
legend_labels = [
    "0 mm (Sin lluvia)",
    "50 mm",
    "100 mm",
    "250 mm",
    "500 mm",
    "750 mm",
    "1000 mm",
    "1250 mm",
    "1500 mm",
    "1750 mm",
    "2000 mm (Máxima precipitación)"
]

legend_colors = [
    "#ffffff", "#b3ccff", "#6699ff", "#3366ff", "#0033ff",
    "#009933", "#33cc33", "#ffff00", "#ff9900", "#ff3300", "#990000"
]

Map.add_legend(title="Precipitación Anual (mm)", labels=legend_labels, colors=legend_colors)

# Crear un control deslizante para seleccionar el año
slider = widgets.IntSlider(min=2003, max=2023, step=1, value=2023, description="Año:")

# Función para actualizar el mapa cuando cambia el slider
def update_map(year):
    Map.layers = []  # Limpiar capas anteriores (NO la leyenda)
    img = chirps_dict[year]  # Obtener la imagen del año seleccionado
    Map.addLayer(img, vis_params, f"Precipitación {year}")

# Conectar el slider con la función
widgets.interactive(update_map, year=slider)

# Mostrar el mapa y el slider
display(slider)
Map

IntSlider(value=2023, description='Año:', max=2023, min=2003)

Map(center=[10, 20], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [34]:
# Cargar datos de población de Nigeria (100m resolución, sin ajuste)
pop_data = ee.Image("WorldPop/GP/100m/pop_age_sex_cons_unadj/NGA_2020")

In [35]:
# Lista de países del Sahel según WorldPop
sahel_countries = ["MRT", "SEN", "MLI", "BFA", "NER", "TCD", "SDN", "SSD", "ERI"]

# Crear una lista de imágenes de población para cada país y seleccionar la banda correcta
pop_images = []
for code in sahel_countries:
    try:
        pop_image = ee.Image(f"WorldPop/GP/100m/pop_age_sex_cons_unadj/{code}_2020").select("population")
        pop_images.append(pop_image)
    except Exception as e:
        print(f"Datos no disponibles para {code}: {e}")

# Fusionar todas las imágenes en una sola
pop_collection = ee.ImageCollection(pop_images).mosaic()

# Definir la región del Sahel (aproximadamente)
region = ee.Geometry.BBox(-18, 5, 30, 20)  # (lon_min, lat_min, lon_max, lat_max)

# Configurar mapa interactivo
Map = geemap.Map()
Map.centerObject(region, 4)
Map.addLayer(pop_collection.clip(region),
             {"min": 0, "max": 500, "palette": ["blue", "yellow", "red"]},
             "Población Sahel 2020")

# Mostrar el mapa
Map

Map(center=[12.78625134285786, 5.999999999999911], controls=(WidgetControl(options=['position', 'transparent_b…

In [36]:
# Cargar fronteras administrativas globales de la ONU (FAO GAUL 2015)
admin_boundaries = ee.FeatureCollection("FAO/GAUL/2015/level0")

# Lista de países del Sahel
sahel_countries = ["Mauritania", "Senegal", "Mali", "Burkina Faso", "Niger", "Chad", "Sudan", "South Sudan", "Eritrea"]

# Filtrar solo los países del Sahel
sahel_boundaries = admin_boundaries.filter(ee.Filter.inList("ADM0_NAME", sahel_countries))

# Configurar mapa interactivo
Map = geemap.Map()
Map.centerObject(sahel_boundaries, 4)
Map.addLayer(sahel_boundaries, {"color": "black"}, "Fronteras del Sahel")

# Mostrar el mapa interactivo
Map

Map(center=[16.584265126758684, 12.349587287037147], controls=(WidgetControl(options=['position', 'transparent…

In [37]:
# 🔹 Definir la región del Sahel
sahel_region = ee.Geometry.BBox(-18, 5, 30, 20)  # Aproximadamente lon_min, lat_min, lon_max, lat_max

# 🔹 Cargar fronteras administrativas para África (FAO GAUL 2015)
admin_boundaries = ee.FeatureCollection("FAO/GAUL/2015/level0")
sahel_countries = ["Mauritania", "Senegal", "Mali", "Burkina Faso", "Niger", "Chad", "Sudan", "South Sudan", "Eritrea"]
sahel_boundaries = admin_boundaries.filter(ee.Filter.inList("ADM0_NAME", sahel_countries))

# 🔹 Cargar datos de población de WorldPop (2020) SOLO en el Sahel
sahel_countries_codes = ["MRT", "SEN", "MLI", "BFA", "NER", "TCD", "SDN", "SSD", "ERI"]
pop_images = []
for code in sahel_countries_codes:
    try:
        pop_image = ee.Image(f"WorldPop/GP/100m/pop_age_sex_cons_unadj/{code}_2020").select("population")
        pop_images.append(pop_image)
    except Exception as e:
        print(f"Datos no disponibles para {code}: {e}")

pop_collection = ee.ImageCollection(pop_images).mosaic().clip(sahel_region)
pop_collection = pop_collection.reproject(crs="EPSG:4326", scale=1000)  # Reducir resolución

# 🔹 Cargar MODIS Land Cover (2020) y recortar al Sahel
modis_landcover = ee.ImageCollection("MODIS/006/MCD12Q1").filter(ee.Filter.calendarRange(2020, 2020, "year")).first().clip(sahel_region)

# 🔹 Cargar CHIRPS optimizado (Precipitación media 2000-2020)
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD").filterDate("2000-01-01", "2020-12-31")
chirps_mean = chirps.mean().select("precipitation")  # Seleccionar la banda correcta
chirps_mean = chirps_mean.clip(sahel_region).reproject(crs="EPSG:4326", scale=10000)  # Reducir resolución

# 🔹 Crear mapa interactivo con África visible pero datos limitados al Sahel
Map = geemap.Map()
Map.centerObject(ee.Geometry.BBox(-25, -35, 55, 40), 3)  # Centrar en África

# Agregar capas optimizadas
Map.addLayer(admin_boundaries, {"color": "gray"}, "Fronteras de África")  # Mostrar todo África en gris
Map.addLayer(sahel_boundaries, {"color": "black"}, "Fronteras del Sahel")  # Solo Sahel en negro
Map.addLayer(pop_collection, {"min": 0, "max": 500, "palette": ["blue", "yellow", "red"]}, "Población Sahel 2020")
Map.addLayer(modis_landcover, {"bands": ["LC_Type1"], "min": 1, "max": 17, "palette": ["green", "brown", "blue"]}, "MODIS Land Cover 2020")
Map.addLayer(chirps_mean, {"min": 0, "max": 1000, "palette": ["white", "blue", "purple"]}, "Precipitación CHIRPS 2000-2020")

# Mostrar el mapa interactivo
Map

Map(center=[2.305331800163843, 14.999999999999785], controls=(WidgetControl(options=['position', 'transparent_…

In [38]:
import ee
import geemap
import ipywidgets as widgets

# Inicializar Google Earth Engine
ee.Initialize(project='buoyant-site-454305-j8')

# 🔹 Definir regiones
AFRICA_BBOX = ee.Geometry.BBox(-25, -35, 55, 40)  # Para visualizar todo África
SAHEL_BBOX = ee.Geometry.BBox(-18, 5, 30, 20)  # Para mostrar solo el Sahel

# 🔹 Lista de países del Sahel y códigos WorldPop
SAHEL_COUNTRIES = ["Mauritania", "Senegal", "Mali", "Burkina Faso", "Niger", "Chad", "Sudan", "South Sudan", "Eritrea"]
SAHEL_CODES = ["MRT", "SEN", "MLI", "BFA", "NER", "TCD", "SDN", "SSD", "ERI"]

# 📌 Función para cargar fronteras del Sahel
def get_sahel_boundaries():
    admin_boundaries = ee.FeatureCollection("FAO/GAUL/2015/level0")
    return admin_boundaries.filter(ee.Filter.inList("ADM0_NAME", SAHEL_COUNTRIES))

# 📌 Función para cargar datos de población WorldPop (2020)
def get_population_data():
    pop_images = []
    for code in SAHEL_CODES:
        try:
            pop_image = ee.Image(f"WorldPop/GP/100m/pop_age_sex_cons_unadj/{code}_2020").select("population")
            pop_images.append(pop_image)
        except Exception as e:
            print(f"Datos no disponibles para {code}: {e}")

    return ee.ImageCollection(pop_images).mosaic().clip(SAHEL_BBOX).reproject(crs="EPSG:4326", scale=1000)

# 📌 Función para cargar datos de MODIS Land Cover con slider interactivo
def get_modis_landcover():
    modis = ee.ImageCollection("MODIS/006/MCD12Q1").select("LC_Type1")
    modis_dict = {year: modis.filter(ee.Filter.calendarRange(year, year, 'year')).first().clip(SAHEL_BBOX) for year in range(2001, 2024)}
    return modis_dict

# 📌 Función para cargar datos de CHIRPS (Precipitación media 2000-2020)
def get_chirps_data():
    chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD").filterDate("2000-01-01", "2020-12-31")
    return chirps.mean().select("precipitation").clip(SAHEL_BBOX).reproject(crs="EPSG:4326", scale=10000)

# 📌 Función para mostrar el mapa interactivo con capas activables
def create_map():
    Map = geemap.Map()
    Map.centerObject(AFRICA_BBOX, 3)  # Centrar en África

    # Cargar datasets
    sahel_boundaries = get_sahel_boundaries()
    pop_collection = get_population_data()
    modis_dict = get_modis_landcover()
    chirps_mean = get_chirps_data()

    # Agregar capas al mapa
    Map.addLayer(sahel_boundaries, {"color": "black"}, "Fronteras del Sahel")
    Map.addLayer(pop_collection, {"min": 0, "max": 500, "palette": ["blue", "yellow", "red"]}, "Población Sahel 2020")
    Map.addLayer(chirps_mean, {"min": 0, "max": 1000, "palette": ["white", "blue", "purple"]}, "Precipitación CHIRPS 2000-2020")

    # 📌 Agregar control deslizante para MODIS
    slider = widgets.IntSlider(min=2001, max=2023, step=1, value=2023, description="Año:")

    def update_modis(year):
        Map.layers = []  # Limpiar capas previas excepto la leyenda
        Map.addLayer(sahel_boundaries, {"color": "black"}, "Fronteras del Sahel")
        Map.addLayer(pop_collection, {"min": 0, "max": 500, "palette": ["blue", "yellow", "red"]}, "Población Sahel 2020")
        Map.addLayer(chirps_mean, {"min": 0, "max": 1000, "palette": ["white", "blue", "purple"]}, "Precipitación CHIRPS 2000-2020")
        Map.addLayer(modis_dict[year], {"bands": ["LC_Type1"], "min": 1, "max": 17, "palette": ["green", "brown", "blue"]}, f"MODIS Land Cover {year}")

    widgets.interactive(update_modis, year=slider)

    display(slider)
    return Map

# 📌 Mostrar el mapa final con controles interactivos
create_map()


IntSlider(value=2023, description='Año:', max=2023, min=2001)

Map(center=[2.305331800163843, 14.999999999999785], controls=(WidgetControl(options=['position', 'transparent_…

In [39]:
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display

# Inicializar Google Earth Engine con el proyecto específico
ee.Initialize(project='buoyant-site-454305-j8')

# 🔹 Definir regiones
AFRICA_BBOX = ee.Geometry.BBox(-25, -35, 55, 40)  # Para visualizar todo África
SAHEL_BBOX = ee.Geometry.BBox(-18, 5, 30, 20)  # Para mostrar solo el Sahel

# 🔹 Lista de países del Sahel y códigos WorldPop
SAHEL_COUNTRIES = ["Mauritania", "Senegal", "Mali", "Burkina Faso", "Niger", "Chad", "Sudan", "South Sudan", "Eritrea"]
SAHEL_CODES = ["MRT", "SEN", "MLI", "BFA", "NER", "TCD", "SDN", "SSD", "ERI"]

# 📌 Función para cargar fronteras del Sahel
def get_sahel_boundaries():
    admin_boundaries = ee.FeatureCollection("FAO/GAUL/2015/level0")
    return admin_boundaries.filter(ee.Filter.inList("ADM0_NAME", SAHEL_COUNTRIES))

# 📌 Función para cargar datos de población WorldPop (2020)
def get_population_data():
    pop_images = []
    for code in SAHEL_CODES:
        try:
            pop_image = ee.Image(f"WorldPop/GP/100m/pop_age_sex_cons_unadj/{code}_2020").select("population")
            pop_images.append(pop_image)
        except Exception as e:
            print(f"⚠️ Datos no disponibles para {code}: {e}")

    return ee.ImageCollection(pop_images).mosaic().clip(SAHEL_BBOX).reproject(crs="EPSG:4326", scale=1000)

# 📌 Función para cargar datos de MODIS Land Cover con validación
def get_modis_landcover():
    modis = ee.ImageCollection("MODIS/006/MCD12Q1").select("LC_Type1")
    modis_dict = {}

    for year in range(2001, 2023):  # Excluir 2023 ya que no hay datos aún
        image = modis.filter(ee.Filter.calendarRange(year, year, 'year')).first()

        if image and image.getInfo() is not None:  # Verifica si la imagen es válida
            modis_dict[year] = image.clip(SAHEL_BBOX)
        else:
            print(f"⚠️ No hay datos de MODIS para el año {year}")

    return modis_dict

# 📌 Función para cargar datos de CHIRPS (Precipitación media 2000-2020)
def get_chirps_data():
    chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD").filterDate("2000-01-01", "2020-12-31")
    return chirps.mean().select("precipitation").clip(SAHEL_BBOX).reproject(crs="EPSG:4326", scale=10000)

# 📌 Función para mostrar el mapa interactivo con capas activables
def create_map():
    # Crear un mapa interactivo centrado en África
    Map = geemap.Map()
    Map.centerObject(AFRICA_BBOX, 3)

    # Cargar datasets
    sahel_boundaries = get_sahel_boundaries()
    pop_collection = get_population_data()
    modis_dict = get_modis_landcover()
    chirps_mean = get_chirps_data()

    # Agregar capas base
    Map.addLayer(sahel_boundaries, {"color": "black"}, "Fronteras del Sahel")
    Map.addLayer(pop_collection, {"min": 0, "max": 500, "palette": ["blue", "yellow", "red"]}, "Población Sahel 2020")
    Map.addLayer(chirps_mean, {"min": 0, "max": 1000, "palette": ["white", "blue", "purple"]}, "Precipitación CHIRPS 2000-2020")

    # 📌 Agregar control deslizante para MODIS (solo si hay datos disponibles)
    available_years = list(modis_dict.keys())  # Solo incluir años con datos
    if not available_years:
        print("⚠️ No hay datos de MODIS disponibles. No se mostrará el control deslizante.")
        return Map

    slider = widgets.IntSlider(min=min(available_years), max=max(available_years), step=1, value=max(available_years), description="Año:")

    # 📌 Función para actualizar MODIS en el mapa
    def update_modis(change):
        Map.layers = Map.layers[:3]  # Mantener las primeras 3 capas y eliminar el resto
        year = slider.value
        modis_layer = modis_dict.get(year, None)  # Obtener la imagen de MODIS si está disponible

        if modis_layer:
            Map.addLayer(modis_layer, {"bands": ["LC_Type1"], "min": 1, "max": 17, "palette": ["green", "brown", "blue"]}, f"MODIS Land Cover {year}")
        else:
            print(f"⚠️ No hay datos de MODIS disponibles para el año {year}")

    slider.observe(update_modis, names='value')  # Ligar el slider con la actualización de MODIS

    display(slider)
    update_modis(None)  # Cargar MODIS inicialmente
    return Map

# 📌 Mostrar el mapa final con controles interactivos
create_map()


⚠️ No hay datos de MODIS para el año 2021
⚠️ No hay datos de MODIS para el año 2022


IntSlider(value=2020, description='Año:', max=2020, min=2001)

Map(center=[2.305331800163843, 14.999999999999785], controls=(WidgetControl(options=['position', 'transparent_…

In [40]:
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display

# Inicializar Google Earth Engine con el proyecto específico
ee.Initialize(project='buoyant-site-454305-j8')

# 🔹 Definir regiones
AFRICA_BBOX = ee.Geometry.BBox(-25, -35, 55, 40)  # Para visualizar todo África
SAHEL_BBOX = ee.Geometry.BBox(-18, 5, 30, 20)  # Para mostrar solo el Sahel

# 🔹 Lista de países del Sahel y códigos WorldPop
SAHEL_COUNTRIES = ["Mauritania", "Senegal", "Mali", "Burkina Faso", "Niger", "Chad", "Sudan", "South Sudan", "Eritrea"]
SAHEL_CODES = ["MRT", "SEN", "MLI", "BFA", "NER", "TCD", "SDN", "SSD", "ERI"]

# 📌 Función para cargar fronteras del Sahel
def get_sahel_boundaries():
    admin_boundaries = ee.FeatureCollection("FAO/GAUL/2015/level0")
    return admin_boundaries.filter(ee.Filter.inList("ADM0_NAME", SAHEL_COUNTRIES))

# 📌 Función para cargar datos de población WorldPop (2020)
def get_population_data():
    pop_images = []
    for code in SAHEL_CODES:
        try:
            pop_image = ee.Image(f"WorldPop/GP/100m/pop_age_sex_cons_unadj/{code}_2020").select("population")
            pop_images.append(pop_image)
        except Exception as e:
            print(f"⚠️ Datos no disponibles para {code}: {e}")

    return ee.ImageCollection(pop_images).mosaic().clip(SAHEL_BBOX).reproject(crs="EPSG:4326", scale=1000)

# 📌 Función para cargar datos de MODIS Land Cover con validación
def get_modis_landcover():
    modis = ee.ImageCollection("MODIS/006/MCD12Q1").select("LC_Type1")
    modis_dict = {}

    for year in range(2001, 2023):  # Excluir 2023 ya que no hay datos aún
        image = modis.filter(ee.Filter.calendarRange(year, year, 'year')).first()

        if image and image.getInfo() is not None:  # Verifica si la imagen es válida
            modis_dict[year] = image.clip(SAHEL_BBOX)
        else:
            print(f"⚠️ No hay datos de MODIS para el año {year}")

    return modis_dict

# 📌 Función para cargar datos de CHIRPS (Precipitación media 2000-2020)
def get_chirps_data():
    chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD").filterDate("2000-01-01", "2020-12-31")
    return chirps.mean().select("precipitation").clip(SAHEL_BBOX).reproject(crs="EPSG:4326", scale=10000)

# 📌 Función para agregar la leyenda
def add_legend(Map):
    legend_dict = {
        "Población Baja (WorldPop)": "#0000FF",  # Azul
        "Población Media": "#FFFF00",  # Amarillo
        "Población Alta": "#FF0000",  # Rojo
        "Precipitación Baja (CHIRPS)": "#FFFFFF",  # Blanco
        "Precipitación Alta": "#800080",  # Morado
        "Bosques (MODIS)": "#008000",  # Verde
        "Áreas Urbanas": "#A52A2A",  # Marrón
        "Agua": "#0000FF"  # Azul
    }

    Map.add_legend(title="Leyenda de Datos", legend_dict=legend_dict)

# 📌 Función para mostrar el mapa interactivo con capas activables
def create_map():
    # Crear un mapa interactivo centrado en África
    Map = geemap.Map()
    Map.centerObject(AFRICA_BBOX, 3)

    # Cargar datasets
    sahel_boundaries = get_sahel_boundaries()
    pop_collection = get_population_data()
    modis_dict = get_modis_landcover()
    chirps_mean = get_chirps_data()

    # Agregar capas base
    Map.addLayer(sahel_boundaries, {"color": "black"}, "Fronteras del Sahel")
    Map.addLayer(pop_collection, {"min": 0, "max": 500, "palette": ["blue", "yellow", "red"]}, "Población Sahel 2020")
    Map.addLayer(chirps_mean, {"min": 0, "max": 1000, "palette": ["white", "blue", "purple"]}, "Precipitación CHIRPS 2000-2020")

    # Agregar leyenda corregida
    add_legend(Map)

    # 📌 Agregar control deslizante para MODIS (solo si hay datos disponibles)
    available_years = list(modis_dict.keys())  # Solo incluir años con datos
    if not available_years:
        print("⚠️ No hay datos de MODIS disponibles. No se mostrará el control deslizante.")
        return Map

    slider = widgets.IntSlider(min=min(available_years), max=max(available_years), step=1, value=max(available_years), description="Año:")

    # 📌 Función para actualizar MODIS en el mapa
    def update_modis(change):
        Map.layers = Map.layers[:3]  # Mantener las primeras 3 capas y eliminar el resto
        year = slider.value
        modis_layer = modis_dict.get(year, None)  # Obtener la imagen de MODIS si está disponible

        if modis_layer:
            Map.addLayer(modis_layer, {"bands": ["LC_Type1"], "min": 1, "max": 17, "palette": ["green", "brown", "blue"]}, f"MODIS Land Cover {year}")
        else:
            print(f"⚠️ No hay datos de MODIS disponibles para el año {year}")

    slider.observe(update_modis, names='value')  # Ligar el slider con la actualización de MODIS

    display(slider)
    update_modis(None)  # Cargar MODIS inicialmente
    return Map

# 📌 Mostrar el mapa final con controles interactivos
create_map()


⚠️ No hay datos de MODIS para el año 2021
⚠️ No hay datos de MODIS para el año 2022


IntSlider(value=2020, description='Año:', max=2020, min=2001)

Map(center=[2.305331800163843, 14.999999999999785], controls=(WidgetControl(options=['position', 'transparent_…

In [41]:
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display

# Inicializar Google Earth Engine con el proyecto específico
ee.Initialize(project='buoyant-site-454305-j8')

# 🔹 Definir regiones
AFRICA_BBOX = ee.Geometry.BBox(-25, -35, 55, 40)  # Para visualizar todo África
SAHEL_BBOX = ee.Geometry.BBox(-18, 5, 30, 20)  # Para mostrar solo el Sahel

# 🔹 Lista de países del Sahel y códigos WorldPop
SAHEL_COUNTRIES = ["Mauritania", "Senegal", "Mali", "Burkina Faso", "Niger", "Chad", "Sudan", "South Sudan", "Eritrea"]
SAHEL_CODES = ["MRT", "SEN", "MLI", "BFA", "NER", "TCD", "SDN", "SSD", "ERI"]

# 📌 Función para cargar fronteras del Sahel
def get_sahel_boundaries():
    admin_boundaries = ee.FeatureCollection("FAO/GAUL/2015/level0")
    return admin_boundaries.filter(ee.Filter.inList("ADM0_NAME", SAHEL_COUNTRIES))

# 📌 Función para cargar datos de población WorldPop (2020)
def get_population_data():
    pop_images = []
    for code in SAHEL_CODES:
        try:
            pop_image = ee.Image(f"WorldPop/GP/100m/pop_age_sex_cons_unadj/{code}_2020").select("population")
            pop_images.append(pop_image)
        except Exception as e:
            print(f"⚠️ Datos no disponibles para {code}: {e}")

    return ee.ImageCollection(pop_images).mosaic().clip(SAHEL_BBOX).reproject(crs="EPSG:4326", scale=1000)

# 📌 Función para cargar datos de MODIS Land Cover con validación
def get_modis_landcover():
    modis = ee.ImageCollection("MODIS/006/MCD12Q1").select("LC_Type1")
    modis_dict = {}

    for year in range(2001, 2023):  # Excluir 2023 ya que no hay datos aún
        image = modis.filter(ee.Filter.calendarRange(year, year, 'year')).first()

        if image and image.getInfo() is not None:  # Verifica si la imagen es válida
            modis_dict[year] = image.clip(SAHEL_BBOX)
        else:
            print(f"⚠️ No hay datos de MODIS para el año {year}")

    return modis_dict

# 📌 Función para cargar datos de CHIRPS (Precipitación media 2000-2020)
def get_chirps_data():
    chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD").filterDate("2000-01-01", "2020-12-31")
    return chirps.mean().select("precipitation").clip(SAHEL_BBOX).reproject(crs="EPSG:4326", scale=10000)

# 📌 Función para agregar la leyenda
def add_legend(Map):
    legend_dict = {
        "Población Baja (WorldPop)": "#0000FF",  # Azul
        "Población Media": "#FFFF00",  # Amarillo
        "Población Alta": "#FF0000",  # Rojo
        "Precipitación Baja (CHIRPS)": "#FFFFFF",  # Blanco
        "Precipitación Alta": "#800080",  # Morado
        "Bosques (MODIS)": "#008000",  # Verde
        "Áreas Urbanas": "#A52A2A",  # Marrón
        "Agua": "#0000FF"  # Azul
    }

    Map.add_legend(title="Leyenda de Datos", legend_dict=legend_dict)

# 📌 Función para crear el mapa y exportarlo a HTML
def create_map():
    # Crear un mapa interactivo
    Map = geemap.Map()
    Map.centerObject(AFRICA_BBOX, 3)

    # Cargar datasets
    sahel_boundaries = get_sahel_boundaries()
    pop_collection = get_population_data()
    modis_dict = get_modis_landcover()
    chirps_mean = get_chirps_data()

    # Agregar capas al mapa
    Map.addLayer(sahel_boundaries, {"color": "black"}, "Fronteras del Sahel")
    Map.addLayer(pop_collection, {"min": 0, "max": 500, "palette": ["blue", "yellow", "red"]}, "Población Sahel 2020")
    Map.addLayer(chirps_mean, {"min": 0, "max": 1000, "palette": ["white", "blue", "purple"]}, "Precipitación CHIRPS 2000-2020")

    # Agregar leyenda
    add_legend(Map)

    # 📌 Guardar el mapa en HTML
    html_file = "mapa_sahel.html"
    Map.to_html(filename=html_file, title="Mapa Interactivo del Sahel")

    print(f"✅ Mapa exportado con éxito: {html_file}")

    return html_file

# 📌 Ejecutar la función para generar el mapa y exportarlo
html_output = create_map()


⚠️ No hay datos de MODIS para el año 2021
⚠️ No hay datos de MODIS para el año 2022
✅ Mapa exportado con éxito: mapa_sahel.html


In [42]:
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display

# Inicializar Google Earth Engine con el proyecto específico
ee.Initialize(project='buoyant-site-454305-j8')

# 🔹 Definir regiones
AFRICA_BBOX = ee.Geometry.BBox(-25, -35, 55, 40)  # Para visualizar todo África
SAHEL_BBOX = ee.Geometry.BBox(-18, 5, 30, 20)  # Para mostrar solo el Sahel

# 🔹 Lista de países del Sahel y códigos WorldPop
SAHEL_COUNTRIES = ["Mauritania", "Senegal", "Mali", "Burkina Faso", "Niger", "Chad", "Sudan", "South Sudan", "Eritrea"]
SAHEL_CODES = ["MRT", "SEN", "MLI", "BFA", "NER", "TCD", "SDN", "SSD", "ERI"]

# 📌 Función para cargar fronteras del Sahel
def get_sahel_boundaries():
    admin_boundaries = ee.FeatureCollection("FAO/GAUL/2015/level0")
    return admin_boundaries.filter(ee.Filter.inList("ADM0_NAME", SAHEL_COUNTRIES))

# 📌 Función para cargar datos de población WorldPop (2020)
def get_population_data():
    pop_images = []
    for code in SAHEL_CODES:
        try:
            pop_image = ee.Image(f"WorldPop/GP/100m/pop_age_sex_cons_unadj/{code}_2020").select("population")
            pop_images.append(pop_image)
        except Exception as e:
            print(f"⚠️ Datos no disponibles para {code}: {e}")

    return ee.ImageCollection(pop_images).mosaic().clip(SAHEL_BBOX).reproject(crs="EPSG:4326", scale=1000)

# 📌 Función para cargar datos de MODIS Land Cover con validación
def get_modis_landcover():
    modis = ee.ImageCollection("MODIS/006/MCD12Q1").select("LC_Type1")
    modis_dict = {}

    for year in range(2001, 2023):  # Excluir 2023 ya que no hay datos aún
        image = modis.filter(ee.Filter.calendarRange(year, year, 'year')).first()

        if image and image.getInfo() is not None:  # Verifica si la imagen es válida
            modis_dict[year] = image.clip(SAHEL_BBOX)
        else:
            print(f"⚠️ No hay datos de MODIS para el año {year}")

    return modis_dict

# 📌 Función para cargar datos de CHIRPS (Precipitación media 2000-2020)
def get_chirps_data():
    chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD").filterDate("2000-01-01", "2020-12-31")
    return chirps.mean().select("precipitation").clip(SAHEL_BBOX).reproject(crs="EPSG:4326", scale=10000)

# 📌 Función para agregar la leyenda
def add_legend(Map):
    legend_dict = {
        "Población Baja (WorldPop)": "#0000FF",  # Azul
        "Población Media": "#FFFF00",  # Amarillo
        "Población Alta": "#FF0000",  # Rojo
        "Precipitación Baja (CHIRPS)": "#FFFFFF",  # Blanco
        "Precipitación Alta": "#800080",  # Morado
        "Bosques (MODIS)": "#008000",  # Verde
        "Áreas Urbanas": "#A52A2A",  # Marrón
        "Agua": "#0000FF"  # Azul
    }

    Map.add_legend(title="Leyenda de Datos", legend_dict=legend_dict)

# 📌 Función para crear el mapa y exportarlo a HTML con MODIS incluido
def create_map():
    # Crear un mapa interactivo
    Map = geemap.Map()
    Map.centerObject(AFRICA_BBOX, 3)

    # Cargar datasets
    sahel_boundaries = get_sahel_boundaries()
    pop_collection = get_population_data()
    modis_dict = get_modis_landcover()
    chirps_mean = get_chirps_data()

    # Agregar capas al mapa
    Map.addLayer(sahel_boundaries, {"color": "black"}, "Fronteras del Sahel")
    Map.addLayer(pop_collection, {"min": 0, "max": 500, "palette": ["blue", "yellow", "red"]}, "Población Sahel 2020")
    Map.addLayer(chirps_mean, {"min": 0, "max": 1000, "palette": ["white", "blue", "purple"]}, "Precipitación CHIRPS 2000-2020")

    # 📌 Agregar MODIS con un año por defecto (último año con datos)
    if modis_dict:
        latest_year = max(modis_dict.keys())  # Último año disponible
        modis_layer = modis_dict[latest_year]
        Map.addLayer(modis_layer, {"bands": ["LC_Type1"], "min": 1, "max": 17, "palette": ["green", "brown", "blue"]}, f"MODIS Land Cover {latest_year}")

    # Agregar leyenda
    add_legend(Map)

    # 📌 Guardar el mapa en HTML
    html_file = "mapa_sahel.html"
    Map.to_html(filename=html_file, title="Mapa Interactivo del Sahel")

    print(f"✅ Mapa exportado con éxito: {html_file} (Incluye MODIS)")

    return html_file

# 📌 Ejecutar la función para generar el mapa y exportarlo
html_output = create_map()


⚠️ No hay datos de MODIS para el año 2021
⚠️ No hay datos de MODIS para el año 2022
✅ Mapa exportado con éxito: mapa_sahel.html (Incluye MODIS)


In [43]:
import pandas as pd

file_path="animales.csv"
df=pd.read_csv(file_path)
df.head()

,date,Country,Moughataa/Department,Commune/Municipality,arvi_threats,evi_threats,ndvi_threats,sipi_threats,SPI_1_threats,SPI_3_threats,SPI_6_threats,SPI_9_threats,SPI_12_threats,Biomass_threats,Nightlights_threats,Global_threats
0,2020-01-01 00:00:00,Mauritania,Aioun,Aioun,Mild threat,No risk,No risk,NaN,No risk,No risk,No risk,Mild threat,Moderate threat,Severe threat,Mild threat,Amenaza severa
1,2020-01-01 00:00:00,Mauritania,Aioun,Bennemane,Moderate threat,Moderate threat,Mild threat,No risk,No risk,No risk,No risk,Mild threat,Moderate threat,Severe threat,Mild threat,Amenaza severa
2,2020-01-01 00:00:00,Mauritania,Aioun,Doueirare,Moderate threat,Mild threat,No risk,No risk,No risk,No risk,No risk,Mild threat,Moderate threat,Severe threat,Mild threat,Amenaza severa
3,2020-01-01 00:00:00,Mauritania,Aioun,Egjert,Mild threat,Moderate threat,No risk,No risk,No risk,No risk,No risk,Mild threat,Moderate threat,Moderate threat,Mild threat,Amenaza moderada
4,2020-01-01 00:00:00,Mauritania,Aioun,N'Savenni,Mild threat,Moderate threat,No risk,No risk,No risk,No risk,No risk,Moderate threat,Moderate threat,Mild threat,Mild threat,Amenaza moderada


In [44]:
df.columns=["Date", "Country", "Wilaya/Region", "Moughataa/Department", "Commune/Municipality", "Cattle_theft_probability"]

# Convert categorical probabilities to numeric values and colors
probability_mapping = {
    "1. Highly unlikely": (1, "#00FF00"),  # Green
    "2. Unlikely": (2, "#FFFF00"),  # Yellow
    "3. Possible": (3, "#FFA500"),  # Orange
    "4. Likely": (4, "#FF0000"),  # Red
    "5. Highly likely": (5, "#8B0000")  # Dark Red
}

df["Theft_Level"] = df["Cattle_theft_probability"].map(lambda x: probability_mapping[x][0])
df["Color"] = df["Cattle_theft_probability"].map(lambda x: probability_mapping[x][1])

# Geocode locations (Approximate lat/lon for Aioun, Mauritania)
location_mapping = {
    "Aioun": (16.6614, -9.6142),  # Latitude, Longitude for Aioun
}

df["Latitude"] = df["Commune/Municipality"].map(lambda x: location_mapping[x][0])
df["Longitude"] = df["Commune/Municipality"].map(lambda x: location_mapping[x][1])

# Create an interactive map
Map = geemap.Map(center=[16.6614, -9.6142], zoom=6)

# Add cattle theft probability points to the map
for _, row in df.iterrows():
    point = ee.Geometry.Point([row["Longitude"], row["Latitude"]])
    feature = ee.Feature(point, {"Theft_Level": row["Theft_Level"]})
    Map.addLayer(feature, {"color": row["Color"]}, f"{row['Cattle_theft_probability']}")

# Display the map
Map

ValueError: Length mismatch: Expected axis has 16 elements, new values have 6 elements

In [ ]:
# Assuming your CSV has 10 columns, you might need to adjust this list.
# Inspect the CSV to get the correct column names if they are different.
df.columns = ["Column1", "Column2", "Column3", "Column4", "Column5",
              "Column6", "Column7", "Column8", "Column9", "Column10"]

# ... rest of your code ...

In [ ]:
# Convert categorical probabilities to numeric values and colors
probability_mapping = {
    "1. Highly unlikely": (1, "#00FF00"),  # Green
    "2. Unlikely": (2, "#FFFF00"),  # Yellow
    "3. Possible": (3, "#FFA500"),  # Orange
    "4. Likely": (4, "#FF0000"),  # Red
    "5. Highly likely": (5, "#8B0000")  # Dark Red
}

df["Theft_Level"] = df["Cattle_theft_probability"].map(lambda x: probability_mapping[x][0])
df["Color"] = df["Cattle_theft_probability"].map(lambda x: probability_mapping[x][1])

# Geocode locations (Approximate lat/lon for Aioun, Mauritania)
location_mapping = {
    "Aioun": (16.6614, -9.6142),  # Latitude, Longitude for Aioun
}

df["Latitude"] = df["Commune/Municipality"].map(lambda x: location_mapping[x][0])
df["Longitude"] = df["Commune/Municipality"].map(lambda x: location_mapping[x][1])

# Create an interactive map
Map = geemap.Map(center=[16.6614, -9.6142], zoom=6)

# Add cattle theft probability points to the map
for _, row in df.iterrows():
    point = ee.Geometry.Point([row["Longitude"], row["Latitude"]])
    feature = ee.Feature(point, {"Theft_Level": row["Theft_Level"]})
    Map.addLayer(feature, {"color": row["Color"]}, f"{row['Cattle_theft_probability']}")

# Display the map
Map